In [1]:
import tensorflow as tf
import os
import numpy as np
import warnings
import cv2
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K

tf.keras.backend.clear_session()

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Default GPU Device: /device:GPU:0
Num GPUs Available:  1


In [2]:
def build_train_data_generator(train_images_path = "D:/GitHub/Datasets/MassachusettsRoads/Train_Drop_Empty/Images",
                               train_masks_path = "D:/GitHub/Datasets/MassachusettsRoads/Train_Drop_Empty/Masks",
                               batch_size = 16,
                               seed = 42,
                               target_size = (256, 256)):

    data_generator_args = dict(featurewise_center = False,
                               samplewise_center = False,
                               featurewise_std_normalization = False,
                               samplewise_std_normalization = False,
                               zca_whitening = False, 
                               zca_epsilon = 1e-06,
                               rotation_range = 90, 
                               width_shift_range = 0.0,
                               height_shift_range = 0.0, 
                               brightness_range = None, 
                               shear_range = 0.0, 
                               zoom_range = 0.0, 
                               channel_shift_range = 0.0,
                               fill_mode = 'nearest', 
                               cval = 0.0,
                               horizontal_flip = True,
                               vertical_flip = True, 
                               rescale = 1/255,
                               preprocessing_function = None,
                               data_format = None,
                               validation_split = 0.0,
                               dtype = None)

    image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(**data_generator_args)
    mask_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(**data_generator_args)

    image_data_generator_ = image_data_generator.flow_from_directory(
            train_images_path,
            batch_size = batch_size,
            target_size = target_size, 
            shuffle = True,
            seed = seed)

    mask_data_generator_ = mask_data_generator.flow_from_directory(
            train_masks_path,
            batch_size = batch_size,
            target_size = target_size, 
            shuffle = True,
            seed = seed)

    train_generator = zip(image_data_generator_, mask_data_generator_)
    
    return train_generator

def build_validation_data_generator(validation_images_path = "D:/GitHub/Datasets/MassachusettsRoads/Validation_Drop_Empty/Images",
                                    validation_masks_path = "D:/GitHub/Datasets/MassachusettsRoads/Validation_Drop_Empty/Masks",
                                    batch_size = 16,
                                    seed = 42,
                                    target_size = (256, 256)):

    data_generator_args = dict(featurewise_center = False,
                               samplewise_center = False,
                               featurewise_std_normalization = False,
                               samplewise_std_normalization = False,
                               zca_whitening = False, 
                               zca_epsilon = 1e-06,
                               rotation_range = 0, 
                               width_shift_range = 0.0,
                               height_shift_range = 0.0, 
                               brightness_range = None, 
                               shear_range = 0.0, 
                               zoom_range = 0.0, 
                               channel_shift_range = 0.0,
                               fill_mode = 'nearest', 
                               cval = 0.0,
                               horizontal_flip = False,
                               vertical_flip = False, 
                               rescale = 1/255,
                               preprocessing_function = None,
                               data_format = None,
                               validation_split = 0.0,
                               dtype = None)

    image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(**data_generator_args)
    mask_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(**data_generator_args)

    image_data_generator_ = image_data_generator.flow_from_directory(
        validation_images_path,
        batch_size = batch_size,
        target_size = target_size, 
        shuffle = True,
        seed = seed)

    mask_data_generator_ = mask_data_generator.flow_from_directory(
        validation_masks_path,
        batch_size = batch_size,
        target_size = target_size, 
        shuffle = True,
        seed = seed)

    validation_generator = zip(image_data_generator_, mask_data_generator_)
    
    return validation_generator

In [3]:
def unet_model(input_size = (256, 256, 3), mult = 1, dropout = 0.1):
    
    inputs = tf.keras.layers.Input(input_size)
    
    # inputs = tf.keras.layers.Lambda(lambda x: x/255)(inputs)
    
    c1 = tf.keras.layers.Conv2D(filters = 16 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(inputs)
    c1 = tf.keras.layers.BatchNormalization()(c1)
    c1 = tf.keras.layers.Dropout(dropout)(c1)
    c1 = tf.keras.layers.Conv2D(filters = 16 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c1)
    c1 = tf.keras.layers.BatchNormalization()(c1)
    p1 = tf.keras.layers.MaxPooling2D(pool_size = (2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(filters = 32 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(p1)
    c2 = tf.keras.layers.BatchNormalization()(c2)
    c2 = tf.keras.layers.Dropout(dropout)(c2)
    c2 = tf.keras.layers.Conv2D(filters = 32 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c2)
    c2 = tf.keras.layers.BatchNormalization()(c2)
    p2 = tf.keras.layers.MaxPooling2D(pool_size = (2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(filters = 64 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(p2)
    c3 = tf.keras.layers.BatchNormalization()(c3)
    c3 = tf.keras.layers.Dropout(dropout)(c3)
    c3 = tf.keras.layers.Conv2D(filters = 64 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c3)
    c3 = tf.keras.layers.BatchNormalization()(c3)
    p3 = tf.keras.layers.MaxPooling2D(pool_size = (2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(filters = 128 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(p3)
    c4 = tf.keras.layers.BatchNormalization()(c4)
    c4 = tf.keras.layers.Dropout(dropout)(c4)
    c4 = tf.keras.layers.Conv2D(filters = 128 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c4)
    c4 = tf.keras.layers.BatchNormalization()(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size = (2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(filters = 256 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(p4)
    c5 = tf.keras.layers.BatchNormalization()(c5)
    c5 = tf.keras.layers.Dropout(dropout)(c5)
    c5 = tf.keras.layers.Conv2D(filters = 256 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c5)
    c5 = tf.keras.layers.BatchNormalization()(c5)

    u6 = tf.keras.layers.Conv2DTranspose(filters = 128 * mult, kernel_size = (2, 2), strides = (2, 2), padding = 'same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4], axis = 3)
    c6 = tf.keras.layers.Conv2D(filters = 128 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(u6)
    c6 = tf.keras.layers.BatchNormalization()(c6)
    c6 = tf.keras.layers.Dropout(dropout)(c6)
    c6 = tf.keras.layers.Conv2D(filters = 128 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c6)
    c6 = tf.keras.layers.BatchNormalization()(c6)

    u7 = tf.keras.layers.Conv2DTranspose(filters = 64 * mult, kernel_size = (2, 2), strides = (2, 2), padding = 'same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3], axis = 3)
    c7 = tf.keras.layers.Conv2D(filters = 64 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(u7)
    c7 = tf.keras.layers.BatchNormalization()(c7)
    c7 = tf.keras.layers.Dropout(dropout)(c7)
    c7 = tf.keras.layers.Conv2D(filters = 64 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c7)
    c7 = tf.keras.layers.BatchNormalization()(c7)

    u8 = tf.keras.layers.Conv2DTranspose(filters = 32 * mult, kernel_size = (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2], axis = 3)
    c8 = tf.keras.layers.Conv2D(filters = 32 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(u8)
    c8 = tf.keras.layers.BatchNormalization()(c8)
    c8 = tf.keras.layers.Dropout(dropout)(c8)
    c8 = tf.keras.layers.Conv2D(filters = 32 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c8)
    c8 = tf.keras.layers.BatchNormalization()(c8)

    u9 = tf.keras.layers.Conv2DTranspose(filters = 16 * mult, kernel_size = (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis = 3)
    c9 = tf.keras.layers.Conv2D(filters = 16 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(u9)
    c9 = tf.keras.layers.BatchNormalization()(c9)
    c9 = tf.keras.layers.Dropout(dropout)(c9)
    c9 = tf.keras.layers.Conv2D(filters = 16 * mult, kernel_size = (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c9)
    c9 = tf.keras.layers.BatchNormalization()(c9)

    outputs = tf.keras.layers.Conv2D(filters = 1, kernel_size = (1, 1), activation = 'sigmoid')(c9)

    model = tf.keras.Model(inputs = [inputs], outputs = [outputs])
    
    # print(model.summary())

    return model

In [4]:
model = unet_model(input_size = (256, 256, 3), mult = 3, dropout = 0.1)
model.compile(optimizer = "adam",
              loss = "binary_crossentropy")

In [5]:
batch_size = 8
train_generator = build_train_data_generator(batch_size = batch_size)
validation_generator = build_validation_data_generator(batch_size = batch_size)
steps_train = np.ceil(len(os.listdir("D:/GitHub/Datasets/MassachusettsRoads/Train_Drop_Empty/Images/Images"))/batch_size)
steps_validation = np.ceil(len(os.listdir("D:/GitHub/Datasets/MassachusettsRoads/Validation_Drop_Empty/Images/Images"))/batch_size)

Found 68788 images belonging to 1 classes.
Found 68788 images belonging to 1 classes.
Found 17198 images belonging to 1 classes.
Found 17198 images belonging to 1 classes.


In [6]:
# tf.keras.backend.clear_session()
# model = unet_model(input_size = (256, 256, 3), mult = 2)
# # model.load_weights("D:/GitHub/Datasets/MassachusettsRoads/model_mult_2_256_crops_drop_empty.h5")
# model.compile(optimizer = "adam",
#               loss = "binary_crossentropy")

In [7]:
epochs = 100
model_path = "D:/GitHub/Datasets/MassachusettsRoads/model_mult_3_256_crops_drop_empty.h5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,
                                                      monitor = "val_loss",
                                                      mode = "min",
                                                      save_best_only = True,
                                                      verbose = 2)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                  min_delta = 0, 
                                                  patience = np.ceil(epochs * 0.05),
                                                  verbose = 2,
                                                  restore_best_weights = True)

history = model.fit_generator(train_generator,
                              validation_data = validation_generator,
                              steps_per_epoch = steps_train,
                              validation_steps = steps_validation,
                              epochs = epochs,
                              callbacks = [model_checkpoint, early_stopping])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
8599/8599 [==============================] - ETA: 0s - loss: 0.1570
Epoch 00001: val_loss improved from inf to 0.11632, saving model to D:/GitHub/Datasets/MassachusettsRoads\model_mult_3_256_crops_drop_empty.h5
8599/8599 [==============================] - 2347s 273ms/step - loss: 0.1570 - val_loss: 0.1163
Epoch 2/100
8599/8599 [==============================] - ETA: 0s - loss: 0.1130
Epoch 00002: val_loss improved from 0.11632 to 0.11086, saving model to D:/GitHub/Datasets/MassachusettsRoads\model_mult_3_256_crops_drop_empty.h5
8599/8599 [==============================] - 2217s 258ms/step - loss: 0.1130 - val_loss: 0.1109
Epoch 3/100
8599/8599 [==============================] - ETA: 0s - loss: 0.1048
Epoch 00003: val_loss improved from 0.11086 to 0.09985, saving model to D:/GitHub/Datasets/MassachusettsRoads\model_mult_3_256_crops_drop_empty.h5
8599/8599 [==============================] - 2215s 258m

In [4]:
tf.keras.backend.clear_session()
model = unet_model(input_size = (256, 256, 3), mult = 3, dropout = 0.1)
model.load_weights("D:/GitHub/Datasets/MassachusettsRoads/model_mult_3_256_crops_drop_empty.h5")
model.compile(optimizer = "adam",
              loss = "binary_crossentropy")

In [5]:
tf.keras.backend.clear_session()

dataset_path = "D:/GitHub/Datasets/MassachusettsRoads"
images = os.path.join(dataset_path, "Images")
masks = os.path.join(dataset_path, "Masks")

# id = 900 # 1000, 2000
id = 127
image = cv2.imread(os.path.join(images, os.listdir(images)[id]))
mask = cv2.imread(os.path.join(masks, os.listdir(masks)[id]))

if image.shape[0] == 1024:
     rows = 4
if image.shape[0] == 1500:
     rows = 6

if image.shape[1] == 1024:
     cols = 4
if image.shape[1] == 1500:
     cols = 6

if rows == 6 & cols == 6:
    image = cv2.resize(image, (rows * 256, cols * 256), interpolation = cv2.INTER_AREA)
    mask = cv2.resize(mask, (rows * 256, cols * 256), interpolation = cv2.INTER_NEAREST)

rows_ = np.round(image.shape[0]/rows)
rows_indices = [0]

for i in np.arange(0, rows):
    if i < rows - 1:
        rows_indices.append(int((i + 1) * rows_))
    else:
        rows_indices.append(int(image.shape[0]))

rows_indices

cols_ = np.round(image.shape[1]/cols)
cols_indices = [0]

for i in np.arange(0, cols):
    if i < cols - 1:
        cols_indices.append(int((i + 1) * cols_))
    else:
         cols_indices.append(int(image.shape[1]))

images_ = []
masks_ = []

for i in np.arange(len(rows_indices) - 1):
    i_start = rows_indices[i]
    i_end = rows_indices[i + 1]

    for j in np.arange(len(cols_indices) - 1):
        j_start = cols_indices[j]
        j_end = cols_indices[j + 1]

        image_ = image[i_start:i_end, j_start:j_end, :]
        images_.append(image_)
        images__ = np.array(images_)
        images__ = images__/255
        mask_ = mask[i_start:i_end, j_start:j_end, :]
        masks_.append(mask_)
        masks__ = np.array(masks_)

prediction_output = np.zeros((image.shape[0], image.shape[1], 1))

prediction = model.predict(images__) * 255

width = len(rows_indices) - 1

for i in np.arange(len(rows_indices) - 1):
    i_start = rows_indices[i]
    i_end = rows_indices[i + 1]

    for j in np.arange(len(cols_indices) - 1):
        j_start = cols_indices[j]
        j_end = cols_indices[j + 1]

        # print(i, j, (i * width) + (j + 1), i_start, i_end, j_start, j_end)
        prediction_output[i_start:i_end, j_start:j_end, :] = prediction[(i * width) + (j + 1) - 1,:, :, :] 

cutoff = np.mean(np.ravel(mask))
pridiction_output_binary = prediction_output > cutoff
pridiction_output_binary = pridiction_output_binary.astype('uint8')
pridiction_output_binary = pridiction_output_binary * 255
pridiction_output_binary = np.concatenate([pridiction_output_binary, pridiction_output_binary, pridiction_output_binary], axis = -1)

image_mask = cv2.addWeighted(image, 1, mask, 1, 1)
image_prediction = cv2.addWeighted(image, 1, pridiction_output_binary, 1, 1)

fig = plt.figure(figsize = (25, 15))
ax1 = fig.add_subplot(2, 3, 1)
ax1.imshow(prediction_output)
ax2 = fig.add_subplot(2, 3, 2)
ax2.imshow(image)
ax3 = fig.add_subplot(2, 3, 3)
ax3.imshow(mask)
ax4 = fig.add_subplot(2, 3, 4)
ax4.imshow(pridiction_output_binary)
ax5 = fig.add_subplot(2, 3, 5)
ax5.imshow(image_prediction)
ax6 = fig.add_subplot(2, 3, 6)
ax6.imshow(image_mask)
plt.show() 

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node functional_1/conv2d/Conv2D (defined at <ipython-input-5-5475c691b139>:67) ]] [Op:__inference_predict_function_2796]

Function call stack:
predict_function
